In [0]:
from pyspark.sql import SparkSession

spark=SparkSession.builder \
    .appName("DeltaVersioningDemo") \
    .getOrCreate()

In [0]:
df_data = [
("Naina", "IT", 90000), # these are tuples, any thing inside () with diffrenent kind od data is tuple.
("John", "HR", 50000),
("Priya", "Finace", 60000),
("Riya", "Finace", 80000),
("Rohit", "IT", 65000),
("Alex", "HR", 85000),]

df_column = ["Name", "Department", "Salary"]

df = spark.createDataFrame(df_data, df_column)
display(df)

In [0]:
df.write.format("delta").mode("overwrite").saveAsTable("employees_delta")

In [0]:
%sql
select * from employees_delta

In [0]:
%sql
describe history employees_delta

In [0]:
%sql
update employees_delta set Salary=100000 where Name="Naina"

In [0]:
%sql
describe history employees_delta

In [0]:
# Merge into--> update a Row
from pyspark.sql import Row

update_data = [Row(Name="Naina", Department="IT", Salary=52000)]
df_update = spark.createDataFrame(update_data)
df_update.createOrReplaceTempView("updates_view")

spark.sql("""
MERGE INTO employees_delta AS target
USING updates_view AS source
ON target.Name  = source.Name
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *
""")


In [0]:
%sql
describe history detail_table

In [0]:
# Schema execution(Add new column)

new_data = [("David", "Finace", 70000, 5)]
new_col = ["Name", "Department", "Salary", "Exprience"]
df_new = spark.createDataFrame(new_data, new_col)
display(df_new)

#Merge schema
df_new.write.format("delta") \
    .option("mergeSchema", true) \
    .mode("append") \
    .saveAsTable("employee_table")

In [0]:
%sql
describe history detal_table

In [0]:
%sql
describe extended detal_table